In [6]:
import json
import numpy as np
import random

# Opening JSON file
f = open('BAG_WFS_build_4326.geojson',)
data = json.load(f)

first = data['features'][0]
# print(data["features"][0])
# print(first)
# print(first["geometry"]['coordinates'][0])

fakes = []
for building in data["features"]:
	fake = {}
	fake["id"] = building["properties"]["identificatie"]
	fake["year"] = building["properties"]["bouwjaar"]
	fake["geometry"] = building["geometry"]

	building_energy_mu = random.randint(5, 30)
	building_energy_sigma = 0.1
	building_energy_increase_constant = 1 / 100 * 30
    
	usage = list(map(lambda x: {
		"year": x,
		"energy": np.random.normal(building_energy_mu + (x - 1922) * building_energy_increase_constant, building_energy_sigma)
		}, range(1922, 2022)))

	fake["energy"] = usage
	fakes.append(fake)

print(fakes[0])
# feature = data["features"]
lowest = min(feature["properties"]["bouwjaar"] for feature in data['features'])
highest = max(feature["properties"]["bouwjaar"] for feature in data['features'])
print(lowest, highest)


for feature in data['features']:
	print(feature["properties"]["bouwjaar"]) 

# Closing file
f.close()

{'id': '0599100000013208', 'year': 1932, 'geometry': {'type': 'Polygon', 'coordinates': [[[4.523021985967453, 51.92077657547447], [4.523023183709508, 51.92077627866481], [4.523040390046194, 51.9207720276727], [4.523103796801922, 51.92075634113758], [4.523110939297128, 51.92075457792882], [4.523121002604995, 51.92075211709586], [4.523129488534569, 51.92075004016897], [4.523147585975468, 51.920745562005905], [4.523164398117351, 51.920741397984926], [4.523154372061224, 51.92072604351117], [4.523159455101577, 51.920724786504664], [4.523215134585583, 51.92071102960806], [4.523233205409467, 51.92073893748721], [4.523214216683051, 51.92074365180674], [4.523236092738546, 51.92077778985136], [4.523159701695709, 51.92079664390733], [4.523190331961749, 51.9208442394592], [4.523177333629125, 51.920847379841504], [4.523184108253133, 51.920857946407], [4.523089750887747, 51.920881252584785], [4.523021985967453, 51.92077657547447]]]}, 'energy': [{'year': 1922, 'energy': 27.894202177027104}, {'year': 